In [21]:
!pip install setfit==1.1.0
!pip install transformers==4.42.2
!mkdir models


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\durek\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\durek\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
A subdirectory or file models already exists.


In [22]:
import pandas as pd
import time
import numpy as np
import torch
import nltk
import collections
from nltk.corpus import stopwords
from setfit import SetFitModel
from datasets import Dataset, load_dataset
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tqdm.pandas()

In [23]:
# dataset from hf_hub
langs = ['java', 'python', 'pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = load_dataset('NLBSE/nlbse25-code-comment-classification')

ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [24]:
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))
stop_words.add(',')

def remove_stop_words(dataset): 
    unstoppered_dataset = []
    for sentence in dataset:
        words = nltk.word_tokenize(sentence)
        filtered_words = [word for word in words if word not in stop_words]
        unstoppered_dataset.append(' '.join(filtered_words))
    return unstoppered_dataset

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\durek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\durek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [26]:
for lan in langs:
    dataset = remove_stop_words(ds[f'{lan}_train']['combo'])
    word_count = dict(collections.Counter(" ".join(dataset).split()))
    tags = [x for x in dataset]
    tag_count = dict(collections.Counter(tags))
    print('total count of words',len(word_count))
    print('total count of tags',len(tag_count))
    most_common_tags = sorted(tag_count.items(), key=lambda x: x[1], reverse=True)[:10]
    most_common_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:10]

    print('Most common tags ', (','.join(tag for tag, _ in most_common_tags)))
    print('Most common words ',  (','.join(word for word, _ in most_common_words)))

total count of words 8855
total count of tags 6759
Most common tags  // $ NON-NLS-1 $ | LanguageSettingsProviderAssociationManager.java,// ok expected . | TestHarFileSystemBasics.java,* @ param fs filesystem | SwiftTestUtils.java,* @ param name name use exception message . | Check.java,// $ NON-NLS-1 $ | MakefileEditor.java,* @ param tUnit translation unit | ASTCache.java,// parse command line | NNThroughputBenchmark.java,@ param owner * Listener variable changes . | PaintTarget.java,@ link CacheLoader | TestingCacheLoaders.java,@ param name * Variable name . | PaintTarget.java
Most common words  |,*,.,@,//,(,),/,param,>
total count of words 2688
total count of tags 1830
Most common tags  traceback recent call last | ConfigDict,brew^func | UseOptimizer,123 | ConfigDict,set 0 fail first retry type . | Retry,1 nan 3 | IntegerArray,versionchanged 025.0 | PlotAccessor,key . | omdict,output input | AdaptiveMaxPool3d,display vid | YouTubeVideo,traceback recent call last | _MockPOP3
Most comm

In [6]:
def get_test_split(dataset_size):
    eighty_split = round(dataset_size * 0.8)
    twenty_split = dataset_size - eighty_split
    return [eighty_split, twenty_split]

def get_train_test(lang):
    X_test = remove_stop_words(ds[f'{lang}_test']['combo'])
    y_test = ds[f'{lang}_test']['labels']
    
    raw_train, raw_val = random_split(ds[f'{lang}_train'], lengths=get_test_split(len(ds[f'{lang}_train'])))

    X_train = remove_stop_words([x['combo'] for x in raw_train])
    y_train = [x['labels'] for x in raw_train]
    
    X_val = remove_stop_words([x['combo'] for x in raw_val])
    y_val = [x['labels'] for x in raw_val]
    return X_test, X_train, X_val, y_test, y_train, y_val

BATCH_SIZE = 32
def get_data_loader(dataset):
    return DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)

In [7]:


class CustomDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

In [8]:


def prepare_data_for_bert(sentences, labels, tokenizer, num_labels):
  tokenized_data = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
  if not isinstance(labels[0], list):
    labels = [[1 if i in label else 0 for i in range(num_labels)] for label in labels]
  tokenized_data["labels"] = torch.tensor(labels, dtype=torch.float32)
  return tokenized_data

def data_collator(features):
  input_ids = torch.stack([f["input_ids"] for f in features])
  attention_mask = torch.stack([f["attention_mask"] for f in features])
  labels = torch.stack([f["labels"] for f in features])
  # print(f"Batch input shape: {input_ids.shape}, Batch labels shape: {labels.shape}")
  return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

for lang in langs:
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  num_labels = len(labels[lang])
  model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
  X_test, X_train, X_val, y_test, y_train, y_val = get_train_test(lang)

  train_encodings = prepare_data_for_bert(X_train, y_train, tokenizer, num_labels)
  val_encodings = prepare_data_for_bert(X_val, y_val, tokenizer, num_labels)
  test_encodings = prepare_data_for_bert(X_test, y_test, tokenizer, num_labels)

  train_dataset = CustomDataset(train_encodings)
  val_dataset = CustomDataset(val_encodings)

  

  training_args = TrainingArguments(
      output_dir='./results',
      num_train_epochs=3,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      warmup_steps=500,
      weight_decay=0.01,
      logging_dir='./logs',
      logging_steps=10,
  )
  
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      data_collator=data_collator
  )

  # Train the model
  trainer.train()
  trainer.model.save_pretrained(f'./models/{lan}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 10/1143 [00:02<05:14,  3.60it/s]

{'loss': 0.6562, 'grad_norm': 2.5559494495391846, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}


  2%|▏         | 20/1143 [00:05<05:07,  3.65it/s]

{'loss': 0.6402, 'grad_norm': 2.3762083053588867, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.05}


  3%|▎         | 30/1143 [00:08<05:05,  3.65it/s]

...

{'loss': 0.3509, 'grad_norm': 1.242890477180481, 'learning_rate': 1.8e-05, 'epoch': 2.77}


 98%|█████████▊| 192/195 [00:16<00:00, 11.75it/s]

{'loss': 0.3203, 'grad_norm': 1.4901281595230103, 'learning_rate': 1.9e-05, 'epoch': 2.92}


100%|██████████| 195/195 [00:17<00:00, 11.07it/s]


{'train_runtime': 17.6151, 'train_samples_per_second': 176.78, 'train_steps_per_second': 11.07, 'train_loss': 0.4557581962683262, 'epoch': 3.0}


In [9]:
total_flops = 0
total_time = 0
scores = []
for lan in langs:
    # to load trained models:
    model = SetFitModel.from_pretrained(f'./models/{lan}')
    # print(f"combo: {len(ds[f'{lan}_test']['combo'])}, label: {len(ds[f'{lan}_test']['labels'])}")
    # to load pretrained models from Hub:
    # model = SetFitModel.from_pretrained(f"NLBSE/nlbse25_{lan}")
    with torch.profiler.profile(with_flops=True) as p:
        begin = time.time()
        for i in range(10):
          y_pred = model(ds[f'{lan}_test']['combo']).numpy().T
        total = time.time() - begin
        total_time = total_time + total
    total_flops = total_flops + (sum(k.flops for k in p.key_averages()) / 1e9)
    y_true = np.array(ds[f'{lan}_test']['labels']).T
    for i in range(len(y_pred)):
        assert(len(y_pred[i]) == len(y_true[i]))
        tp = sum([true == pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
        tn = sum([true == pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
        fp = sum([true == 0 and pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
        fn = sum([true == 1 and pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*tp) / (2*tp + fp + fn)
        scores.append({'lan': lan, 'cat': labels[lan][i],'precision': precision,'recall': recall,'f1': f1})
        print({'lan': lan, 'cat': labels[lan][i],'precision': precision,'recall': recall,'f1': f1})
print("Compute in GFLOPs:", total_flops/10)
print("Avg runtime in seconds:", total_time/10)
scores = pd.DataFrame(scores)
scores

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './models/java'. Use `repo_type` argument if needed.

In [10]:
print("Compute in GFLOPs:", total_flops/10)
print("Avg runtime in seconds:", total_time/10)
scores = pd.DataFrame(scores)

max_avg_runtime = 5
max_avg_flops = 5000

def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = scores.f1.mean()
avg_runtime = total_time/10
avg_flops = total_flops/10

round(score(avg_f1, avg_runtime, avg_flops), 2)

Compute in GFLOPs: 0.0
Avg runtime in seconds: 0.0


AttributeError: 'DataFrame' object has no attribute 'f1'